In [32]:
from hmmlearn import hmm
from abc_utils import *
import numpy as np
from tqdm import tqdm
RANDOM_STATE = 0

In [33]:
train_set, _ = load_harmonization_train_test()

In [34]:
abc_texts = [train_set.sample(1, random_state=RANDOM_STATE)['output'].item() for _ in range(1000)]

In [24]:
# list_o_dfs = [abc_to_dataframe(abc_text) for abc_text in tqdm(abc_texts)]

In [35]:
l_o_df = []
l_o_df_lengths = []
for abc_text in abc_texts:
    df = abc_to_dataframe(abc_text)
    lengths = len(df)
    l_o_df.append(df)
    l_o_df_lengths.append(lengths)

In [26]:
print(abc_text)

T:Train tune 521761
X:521761
E:8
L:1/4
Q:1/4=160
M:4/4
K:G
D |"G" G G/B/ d B | g d B G |"Am" c A/B/ c (3B/c/B/ |"D7" A G F D |"G" G B/B/ d B | g d B G | 
"Am" A A/B/ c e/c/ |"D7" A F"G" G :: D |"G" G g/g/ g g/f/ |"C" e e/e/ e e/d/ |"Am" c a a a/g/ | 
"D" f d d e/f/ |"Em" g e"A7" e a/g/ |"D" f d"G" d g |"A7" e ^c"D7" A d/=c/ |"G" B G G :|


In [27]:
l_o_df[1]

,measure,beat,chord,melody
0,0,4.0,0,70
1,1,1.0,1,75
2,1,2.0,1,75
3,1,2.5,1,79
4,1,3.0,1,82
...,...,...,...,...
73,16,3.0,8,77
74,16,4.0,8,82
75,16,4.5,8,80
76,17,1.0,1,79


In [28]:
music_df = pd.concat(l_o_df)
music_df.replace('-', 0, inplace=True)
print(music_df['melody'].value_counts())
print(music_df.head())
state_chords = music_df.chord.values
obs_melody = music_df.melody.values
symbols = sorted(list(set(obs_melody)))
print(len(symbols), symbols)


melody
79    12000
82    10000
87    10000
84    10000
75     9000
80     7000
77     6000
86     4000
89     4000
70     3000
74     2000
81     1000
Name: count, dtype: int64
   measure beat  chord  melody
0        0  4.0      0      70
1        1  1.0      1      75
2        1  2.0      1      75
3        1  2.5      1      79
4        1  3.0      1      82
12 [70, 74, 75, 77, 79, 80, 81, 82, 84, 86, 87, 89]


In [29]:
h = hmm.CategoricalHMM(n_components=5, n_iter=1000, tol=1e-6)
h.fit(obs_melody.reshape(-1, 1), lengths=l_o_df_lengths)
pi = h.startprob_
A = h.transmat_.T
B = h.emissionprob_.T

In [30]:
B.shape

(90, 5)

In [31]:
for i in range(len(B)):
    print(u"{}, {:0.4f}, {:0.4f}, {:0.4f}, {:0.4f}, {:0.4f}".format(symbols[i], *B[i,:]))

70, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000
74, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000
75, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000
77, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000
79, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000
80, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000
81, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000
82, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000
84, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000
86, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000
87, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000
89, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000


IndexError: list index out of range

In [55]:
np.sum(h.emissionprob_.T, axis=0)

array([1., 1., 1., 1., 1.])